In [9]:
import os
import sys
import math
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.preprocessing import MinMaxScaler
#https://tech.zms.co.jp/time-series-prediction/

In [53]:
def preprocess():
    current_path = os.path.join(Path().resolve(),"C:/Users/cre/Desktop/git/xauusd_predict/")
    df = pd.read_csv(current_path + 'xauusd.csv', header=0)
    for i, d in enumerate(df["Date"]):
        if len(df["Date"][i].split("/")[1]) != 2 and len(df["Date"][i].split("/")[2]) != 2:
            df["Date"][i] = df["Date"][i].split("/")[0] + "/0" + df["Date"][i].split("/")[1] +"/0"+df["Date"][i].split("/")[2]
        elif len(df["Date"][i].split("/")[1]) != 2:
            df["Date"][i] = df["Date"][i].split("/")[0] + "/0" + df["Date"][i].split("/")[1] +"/"+df["Date"][i].split("/")[2]
        elif len(df["Date"][i].split("/")[2]) != 2:
            df["Date"][i] = df["Date"][i].split("/")[0] + "/" + df["Date"][i].split("/")[1] +"/0"+df["Date"][i].split("/")[2]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')

    original_data = df.values

    return original_data

In [54]:
def create_model(hidden_size, batch_size=None, time_steps=None, stateful=False):

    inputs = Input(batch_shape=(batch_size, None, 1))
    x = inputs
    x = LSTM (hidden_size, stateful=stateful, return_sequences=True)(x)
    x = Dropout(0.5)(x)
    x = LSTM (hidden_size, stateful=stateful, return_sequences=False)(x)
    x = Dropout(0.5)(x)
    x = Dense(1, kernel_regularizer=l2(0.0001), activation='relu')(x)
    outputs = x

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [55]:
# hyper parameters
training = True     # 訓練モード or 予測モード
train_rate = 0.8    # 訓練データの割合
time_steps = 100    # 一度の予測に使うデータ数
predict_day = 30    # 予測する日数
hidden_size = 64    # 隠れ層の大きさ
batch_size = 64     # バッチサイズ
epochs = 500        # エポック数

In [56]:
# データの読み込み
from pathlib import Path
current_path = os.path.join(Path().resolve(),"C:/Users/cre/Desktop/git/xauusd_predict/")
original_data = preprocess()       # (3871, 1)

# 訓練データとテストデータの分割と正規化   
border = math.floor(len(original_data) * train_rate)
train = original_data[:border]     # (3096, 1)
test  = original_data[border:]     # ( 775, 1)

train_scaler = MinMaxScaler(feature_range=(0, 1))
train_scaler.fit(train)                         

scaled_train = train_scaler.transform(train)
scaled_test  = train_scaler.transform(test)    # テストデータを正規化の基準に入れてはならない

C:\Users\cre\AppData\Local\Temp\ipykernel_7700\2712140321.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"][i] = df["Date"][i].split("/")[0] + "/0" + df["Date"][i].split("/")[1] +"/0"+df["Date"][i].split("/")[2]
C:\Users\cre\AppData\Local\Temp\ipykernel_7700\2712140321.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"][i] = df["Date"][i].split("/")[0] + "/0" + df["Date"][i].split("/")[1] +"/"+df["Date"][i].split("/")[2]
C:\Users\cre\AppData\Local\Temp\ipykernel_7700\2712140321.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [103]:
# 学習モード
if training:
    # 入力データとラベルデータの作成
    data_size = len(scaled_train) - time_steps - predict_day        # 2940 
    print(data_size)     
    print(time_steps) # 2940
    x_train = np.zeros((data_size, time_steps))                    # (2940, 100)
    y_train = np.zeros((4, data_size))                             # (2940,   1)
    
    for i in range(data_size):
        x_train[i] = scaled_train[i:i + time_steps].T              # (2940, 100)
        y_train[i] = scaled_train[i + time_steps + predict_day]    # (2940,   1)

    data_size = len(scaled_test) - time_steps - predict_day        # 645
    x_test = np.zeros((data_size, time_steps))                     # (645, 100)
    y_test = np.zeros((data_size, 1))                              # (645,   1)
    for i in range(data_size):
        x_test[i] = scaled_test[i:i + time_steps].T                # (645, 100)
        y_test[i] = scaled_test[i + time_steps + predict_day]      # (645,   1)

    # モデルの入力に形状を合わせる
    x_train = x_train.reshape(len(x_train), time_steps, 1)  # (2940, 100, 1)
    x_test  = x_test.reshape(len(x_test), time_steps, 1)    # ( 645, 100, 1)
    y_train = y_train.reshape(len(y_train), 1, 1)           # (2940,   1, 1)
    y_test  = y_test.reshape(len(y_test), 1, 1)             # ( 645,   1, 1)

2490
100


ValueError: could not broadcast input array from shape (4,100) into shape (100,)